In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data_for_ML_NO2_24hr_0.1.txt", sep=",", header=None) # load the preprocessed data
#print(df.shape)
#df.head()

df.columns = ["date", "lon", "lat", "DOW", "Season", "Tropomi NO2", "RD","WS", "WD", "T", "RH", "BLH", "SP", "T2m", "DUV", "E", "EVI", "NDVI", "insitu_no2"]
df_new = df.drop([], axis=1)
df_new.dropna(inplace=True)
#print(df_new.shape)
#df_new.head()

In [ ]:
x = df_new.iloc[:, 3:-1] # input features
y = df_new.iloc[:, -1] # target feature

from sklearn.model_selection import train_test_split # train and test splilt for 70/30 validation and hyper parameter tuning
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.3)

In [ ]:
## hyper parameter tuning

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

#cs_cv_hy_pa = RandomizedSearchCV(XGBRegressor(), {'max_depth': range(0, 20, 2), 'learning_rate': np.arange(0, 0.5, 0.05), 'reg_lambda': range(0, 20,2), 'reg_alpha': range(0, 30,2), 'gamma': range(0, 31, 2), 'min_child_weight': range(0, 11, 2), 'n_estimators': range(0,3000,250)}, cv=5, n_iter=5000)
cs_cv_hy_pa = GridSearchCV(XGBRegressor(), {'n_estimators': range(0,3000,250)}, cv=5)

cs_cv_hy_pa.fit(x_train, y_train)
print(pd.DataFrame(cs_cv_hy_pa.cv_results_))
print(cs_cv_hy_pa.best_score_)
print(cs_cv_hy_pa.best_params_)

In [ ]:
## 70/30 evaluation 
from xgboost import XGBRegressor
xg_reg = XGBRegressor(max_depth=10, learning_rate=0.3, reg_lambda=12, reg_alpha=18, gamma=20, min_child_weight=16, n_estimators=2500)

xg_reg.fit(x_train,y_train)
y_pred = xg_reg.predict(x_test)
print(y_pred.shape)

import matplotlib.pyplot as plt

m, b = np.polyfit(y_test, y_pred, 1)

fig_dims = (10, 10)
fig, ax = plt.subplots(figsize=fig_dims)

plt.scatter(y_test, y_pred, s=100, edgecolors='black')
plt.plot(y_test, m*y_test + b,color='red',linewidth=5)
plt.grid(True,alpha=0.3)
plt.plot([0, 250], [0, 250], 'r--',linewidth=5)
plt.xlabel('Ground-truth $NO_2$ (µg m$^{-3}$)', fontsize=30)
plt.ylabel('Predicted $NO_2$ (µg m$^{-3}$)', fontsize=30)
plt.xlim([0, 120])
plt.ylim([0, 120])
plt.xticks([0, 50, 100],fontsize=30)
plt.yticks([50, 100],fontsize=30)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print ("R^2: ", r2)
print ("RMSE: ", mse**(1/2))
print ("slope: ", m)

In [ ]:
# Feature importance_SHAP values
import shap
explainer = shap.TreeExplainer(xg_reg)
shap_values = explainer.shap_values(x_train)

shap_values.shape

plt.grid(True,alpha=0.3)
shap.summary_plot(shap_values, x_train, plot_type="bar", plot_size=[10,10], max_display=10)

In [ ]:
# Feature importance_tree
import matplotlib.pyplot as plt
from xgboost import plot_importance

sorted_idx = xg_reg.feature_importances_.argsort()

fig_dims = (8, 8)
fig, ax = plt.subplots(figsize=fig_dims)

plt.barh(x.columns[sorted_idx], xg_reg.feature_importances_[sorted_idx])
plt.xlabel('Feature importance', fontsize=20)
plt.grid(True,alpha=0.3)
plt.xticks([0, 0.25, 0.5, 0.75, 1], fontsize=20)
plt.yticks(fontsize=20)

In [ ]:
# Random 5-fold CV

from xgboost import XGBRegressor
xg_reg = XGBRegressor(max_depth=10, learning_rate=0.3, reg_lambda=12, reg_alpha=18, gamma=20, min_child_weight=16, n_estimators=2500)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

df_new_2 = df_new.sample(frac=1)
x = df_new_2.iloc[:, 3:-1]
y = df_new_2.iloc[:, -1]

rmse_blo_ran_sam_app = []
r2_blo_ran_sam_app = []

for i in range(0,5):
    x_test = x.iloc[round(i*0.2*x.shape[0]):round((i+1)*0.2*x.shape[0]), :]
    y_test = y.iloc[round(i*0.2*y.shape[0]):round((i+1)*0.2*y.shape[0])]
    
    x_train = x
    x_train = x_train.drop (x_train.index[range(round(i*0.2*x.shape[0]),round((i+1)*0.2*x.shape[0]))])
    
    y_train = y
    y_train = y_train.drop (y_train.index[range(round(i*0.2*y.shape[0]),round((i+1)*0.2*y.shape[0]))])

    xg_reg.fit(x_train,y_train)
    y_pred = xg_reg.predict(x_test)
    
    rmse_cv = (mean_squared_error(y_test, y_pred))**(1/2) 
    r2_cv = r2_score(y_test, y_pred)
    rmse_blo_ran_sam_app.append(rmse_cv)
    r2_blo_ran_sam_app.append(r2_cv)

print(np.mean(r2_blo_ran_sam_app))
print(np.std(r2_blo_ran_sam_app))

print(np.mean(rmse_blo_ran_sam_app))
print(np.std(rmse_blo_ran_sam_app))

In [ ]:
# time-leave-out 5-fold CV

from xgboost import XGBRegressor
xg_reg = XGBRegressor(max_depth=10, learning_rate=0.3, reg_lambda=12, reg_alpha=18, gamma=20, min_child_weight=16, n_estimators=2500)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

rmse_blo_temp_sam_app = []
r2_blo_temp_sam_app = []

for i in range(0,5):
    x_test = x.iloc[round(i*0.2*x.shape[0]):round((i+1)*0.2*x.shape[0]), :]
    y_test = y.iloc[round(i*0.2*y.shape[0]):round((i+1)*0.2*y.shape[0])]
    
    x_train = x
    x_train = x_train.drop (x_train.index[range(round(i*0.2*x.shape[0]),round((i+1)*0.2*x.shape[0]))])
    
    y_train = y
    y_train = y_train.drop (y_train.index[range(round(i*0.2*y.shape[0]),round((i+1)*0.2*y.shape[0]))])

    xg_reg.fit(x_train,y_train)
    y_pred = xg_reg.predict(x_test)
    
    rmse_cv = (mean_squared_error(y_test, y_pred))**(1/2) 
    r2_cv = r2_score(y_test, y_pred)
    rmse_blo_temp_sam_app.append(rmse_cv)
    r2_blo_temp_sam_app.append(r2_cv)

print(np.mean(r2_blo_temp_sam_app))
print(np.std(r2_blo_temp_sam_app))

print(np.mean(rmse_blo_temp_sam_app))
print(np.std(rmse_blo_temp_sam_app))

In [ ]:
# location-leave-out 5-fold CV

from xgboost import XGBRegressor
xg_reg = XGBRegressor(max_depth=10, learning_rate=0.3, reg_lambda=12, reg_alpha=18, gamma=20, min_child_weight=16, n_estimators=2500)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

rmse_spa_loc_sam_app = []
r2_spa_loc_sam_app = []

lat_min = 47
div = 1.7

for i in range(0,5):
    
    df_test = df_new.loc[(df_new['lat'] >= lat_min) & (df_new['lat'] <= lat_min+div)]
    x_test = df_test.iloc[:, 3:-1]
    y_test = df_test.iloc[:, -1]
    
    df_train = df_new.drop(df_new[(df_new['lat'] >= lat_min) & (df_new['lat'] <= lat_min+div)].index)
    x_train = df_train.iloc[:, 3:-1]
    y_train = df_train.iloc[:, -1]
    
    lat_min += div
    
    xg_reg.fit(x_train,y_train)
    y_pred = xg_reg.predict(x_test)
    
    rmse_cv = (mean_squared_error(y_test, y_pred))**(1/2) 
    r2_cv = r2_score(y_test, y_pred)
    rmse_spa_loc_sam_app.append(rmse_cv)
    r2_spa_loc_sam_app.append(r2_cv)

print(np.mean(r2_spa_loc_sam_app))
print(np.std(r2_spa_loc_sam_app))

print(np.mean(rmse_spa_loc_sam_app))
print(np.std(rmse_spa_loc_sam_app))
